### Welcome to zachFeeds - the Ultimate RSS Parser!

- zachFeeds takes in any given RSS Feed, and stores it into FireBase, allowing you to star your favorites, or just compile articles from hundreds of different feeds

- zachFeeds is written in Flask, a lightweight Python webserver, with Jinja2 and HTML as the frontend markup. The reason I did not use React or Angular is that both of those require running some version of Node.js on the host machine, and this is designed to be completely platform-agnostic, without the need for Docker!

- Flask was better than Django for this project because I did not need the heavy extra features of Django, and no complex routing or host-machine SQL is needed!

### How the backend works

- In order to get data, I have one continuous table in Firebase of RSS XML (XML is a markup language) links, that get iterated through every time the 'Feed' is updated!

- In order to get the data from these links, I use a Python module called requests, which will return the XML of every feed. I then [PARSE DATA INSTRUCTIONS HERE]

- Once it is parsed into a dict, I upload the data from each article into my Firebase DB, and refresh the frontend.


### How the frontend works

- The frontend is written in Jinja2, an in-markup scripting language. 

- The main (home) route, written in the flask routes file, just pulls data from firebase, turns it into a dict, then passes it to Jinja, where I can iterate over each object, and insert it into my HTML table for the user to see. 

- The update button is what tells Flask to update Firebase with new articles.

### Minimum Viable Product: 

- Use requests to parse XML into my dict, and post it to Firebase, then retrieve that data

### See below for how most of the program logic works! The rest of the app is just Flask boilerplate, but this is the important bit.

In [2]:
#MVP imports
import requests #Get/Post data to and from web servers
import xmltodict #Parse raw XML data from RSS Feeds
import firebase_admin
from firebase_admin import credentials, db

In [7]:
# Get data from RSS Feed (In this case Google Alerts feed for COVID)

def getRawData(dataLink): #Get raw data
    return requests.get(dataLink).text #Requests returns raw XML data

def convertRawData(dataLink): #Convert raw XML to Dict
    rawData = getRawData(dataLink) #Get raw XML Data
    rawDict =  xmltodict.parse(rawData, dict_constructor=dict) #Convert to OrderedDict
    
    convertedDict = rawDict["feed"] #Root element of RSS Feed
    
    return convertedDict

def rawDataToDict(rawData): #Convert all the raw data into something usable
    finalDict = {
        "title": rawData["title"],
        "entries": []
    }
    
    for article in rawData["entry"]: #Entry is an array in the dict
        print(article["published"])
#         articleDict = { #Get necessary elements from current entry
#             "title": article["title"]["#text"], 
#             "link": article["link"]["@href"],
#             "content": article["content"]['#text']
#         }
        
#         finalDict["entries"].append(articleDict) #Append current entry to the data
        
    return finalDict
        
# TEST WITH GOOGLE ALERT FEED
data = convertRawData("https://www.google.com/alerts/feeds/00698824683610429194/1098921106065488311")

current_data = rawDataToDict(data)

2020-11-24T18:51:49Z
2020-11-24T17:37:30Z
2020-11-24T17:03:02Z
2020-11-24T15:56:15Z
2020-11-24T15:45:00Z
2020-11-24T15:15:37Z
2020-11-24T15:11:15Z
2020-11-24T15:09:40Z
2020-11-24T15:00:00Z
2020-11-24T13:52:52Z
2020-11-24T13:32:00Z
2020-11-24T13:18:45Z
2020-11-24T13:18:45Z
2020-11-24T13:03:27Z
2020-11-24T12:56:15Z
2020-11-24T12:03:26Z
2020-11-24T12:00:00Z
2020-11-24T12:00:00Z
2020-11-24T11:25:24Z
2020-11-24T11:04:36Z


In [4]:
# Put data from dict into FireBase
cred = credentials.Certificate('firebaseKey.json') #Service Account for DB Access

firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://zachfeeds-86449.firebaseio.com/'
})

# ref = db.reference('articles') #Put articles

# for article in current_data["entries"]: #For each article
#     new_article_ref = ref.push({ #Post the data
#         'title': article['title'],
#         'link': article['link'],
#         'content': article['content'],
#     })




In [57]:
#Get data out of firebase

ref = db.reference('articles') #get articles data

articles = ref.get()

# print(articles)

storedArticles = [] #Array of stored articles to later iterate over

for article in articles: #For article in FB return
    currentArticle = {} 
    for elem in articles[article]: #For each element of the article
        currentArticle[elem] = articles[article][elem] #Append data to the dict
    storedArticles.append(currentArticle) #Append the dict to the array
    
print(storedArticles[0]['title']) #Test data in array

8 months after Iowa&#39;s first <b>COVID</b>-19 death, toll at 2224


In [6]:
#Add feeds to firebase

ref = db.reference('feeds') #get feeds data

new_feed_ref = ref.push("https://www.google.com/alerts/feeds/00698824683610429194/3310492921969815257")